In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.colors import ListedColormap
from mpl_toolkits.mplot3d import Axes3D
from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, chi2, f_classif
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import Perceptron
from sklearn.svm import SVC, LinearSVC
from sklearn.neural_network import MLPClassifier

## Load Data:

In [2]:
train = pd.read_csv("train.csv", header=0)
test = pd.read_csv("test.csv", header=0)
validate = pd.read_csv("validate.csv", header=0)

In [7]:
train.dtypes

Vote                                                     object
Occupation_Satisfaction                                 float64
Avg_monthly_expense_when_under_age_21                   float64
AVG_lottary_expanses                                    float64
Avg_Satisfaction_with_previous_vote                     float64
Garden_sqr_meter_per_person_in_residancy_area           float64
Financial_balance_score_(0-1)                           float64
%Of_Household_Income                                    float64
Avg_government_satisfaction                             float64
Avg_education_importance                                float64
Avg_environmental_importance                            float64
Avg_Residancy_Altitude                                  float64
Yearly_ExpensesK                                        float64
%Time_invested_in_work                                  float64
Yearly_IncomeK                                          float64
Avg_monthly_expense_on_pets_or_plants   

## Arrange Classifiers:

In [ ]:
names = ["Nearest Neighbors", "Naive Bayes", "Decision Tree", "Perceptron", 
         "Linear SVM OVO", "Linear SVM OVR", "MLP"] 

In [ ]:
classifiers = [
    KNeighborsClassifier(1),
    GaussianNB(),
    DecisionTreeClassifier(max_depth=5),
    Perceptron(n_iter=100),
    SVC(kernel="linear", C=1),
    LinearSVC(C=1),
    MLPClassifier(verbose=0, activation='relu', hidden_layer_sizes=(50, 25, 10), 
                  random_state=0, max_iter=500, solver='sgd', 
                  learning_rate='invscaling', momentum=.9,
                  nesterovs_momentum=True, learning_rate_init=0.2)]
    

In [ ]:
# Create color maps
cmap_light = ListedColormap(['#AAAAFF', '#AAFFAA', '#FFAAAA'])
cmap_bold = ListedColormap(['#0000FF', '#00FF00', '#FF0000'])

In [ ]:
figure = plt.figure(3, figsize=(21, 9))
plt.clf()

## Run over classifiers:

In [ ]:
i = 1
# iterate over classifiers
for name, clf in zip(names, classifiers):
    ax = plt.subplot(len(datasets), len(classifiers) + 1, i)
    clf.fit(X_train, y_train)
    score = clf.score(X_test, y_test)

        # Plot the decision boundary. For that, we will assign a color to each
        # point in the mesh [x_min, m_max]x[y_min, y_max].
        Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])

        # Put the result into a color plot
        Z = Z.reshape(xx.shape)
        ax.contourf(xx, yy, Z, cmap=cmap_light, alpha=.8)

        # Plot also the training points
        ax.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap=cmap_bold, alpha=0.5)
        # and testing points
        ax.scatter(X_test[:, 0], X_test[:, 1], c=y_test, cmap=cmap_bold)

        ax.set_xlim(xx.min(), xx.max())
        ax.set_ylim(yy.min(), yy.max())
        ax.set_xticks(())
        ax.set_yticks(())
        ax.set_title(name)
        ax.text(xx.max() - .3, yy.min() + .3, ('%.2f' % score).lstrip('0'),
                size=15, horizontalalignment='right')
        i += 1

figure.subplots_adjust(left=.001, right=.999)
plt.show()